# Below code is for face mask detection in the images

In [3]:
import cv2
import numpy as np

net = cv2.dnn.readNet('yolov3_training_last.weights', 'yolov3_testing.cfg')

classes = []
with open("classes.txt", "r") as f:
    classes = f.read().splitlines()


font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(100, 3))

#reading the image
img = cv2.imread('00.jpg')      

height, width, channels = img.shape

blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
net.setInput(blob)
output_layers_names = net.getUnconnectedOutLayersNames()
layerOutputs = net.forward(output_layers_names)

boxes = []
confidences = []
class_ids = []

for output in layerOutputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.2:
            center_x = int(detection[0]*width)
            center_y = int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)

            x = int(center_x - w/2)
            y = int(center_y - h/2)

            boxes.append([x, y, w, h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)

#using Non-maximum suppression and getting the object identified 
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.2, 0.4)

#using cv2 to draw reactangles around the objects identified
if len(indexes)>0:
    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence = str(round(confidences[i],2))
        #color = (0, 255, 0) if label == "with_mask" else (0, 0, 255)
        #color = colors[i]
        color = (0, 255, 0) if label == "with_mask" else (0, 0, 255)
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        #cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255,255,255), 2)
        
cv2.imshow('Image',img)
#cv2.imwrite('new1.png',img)
cv2.waitKey(0)
cv2.destroyAllWindows() 

# Use below code, when using the videos

In [2]:
import cv2
import numpy as np


net = cv2.dnn.readNet('yolov3_training_last.weights', 'yolov3_testing.cfg')

classes = []
with open("classes.txt", "r") as f:
    classes = f.read().splitlines()

cap = cv2.VideoCapture("2.mp4") #when using web cam use 0 as the parameters
                                #for saved videos use the path of teh videos as parameter e.g "path/ayaz/video.mp4"
                            

font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(100, 3))

#choosing the saved video frame sizes 
frame_width = int(cap.get(3)) 
frame_height = int(cap.get(4)) 
   
size = (frame_width, frame_height) 

#used for saving teh video file and th3 format is specified

result = cv2.VideoWriter('videofilename.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, size) 

while True:
    ret, img = cap.read()
    height, width, channels = img.shape
    
    #preprocessing the frames
    blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)
    #using Non-maximum suppression and getting the object identified 
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.2, 0.4)
    
    #using cv2 to draw reactangles around the objects identified
    if len(indexes)>0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i],2))
            #color = colors[i]
            color = (0, 255, 0) if label == "with_mask" else (0, 0, 255)
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
            #cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255,255,255), 2)
    
    result.write(img)
    #uncomment the below line to show the video processing in real time 
    #cv2.imshow('Image', img)
    key = cv2.waitKey(1)
    #press esc key to exit the cv2.video showing window
    if key==27:
        break

cap.release()
result.release()
cv2.destroyAllWindows()